In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import dask.dataframe as dd
import streamlit as st
import numpy as np

In [102]:
diag_cml_lab = pd.read_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\diag_cml_lab')
diag_cml_lab = diag_cml_lab.drop('Unnamed: 0', axis = 1)
diag_cml_lab

,subject_id,has_cvd,duration_cvd,icd_code,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary,label,valuenum,valueuom
0,10035631,1,0.000000,I248,0,0,0,1,0,0,Absolute Basophil Count,0.00,K/uL
1,10035631,1,0.000000,I248,0,0,0,1,0,0,Absolute Neutrophil Count,3.15,K/uL
2,10035631,1,0.000000,I248,0,0,0,1,0,0,Metamyelocytes,1.00,%
3,10035631,1,0.000000,I248,0,0,0,1,0,0,Metamyelocytes,0.00,%
4,10035631,1,0.000000,I248,0,0,0,1,0,0,Immunoglobulin A,127.00,mg/dL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093897,19912102,1,-58.541393,I259,0,0,0,1,0,0,Sodium,134.00,mEq/L
2093898,19912102,1,-58.541393,I259,0,0,0,1,0,0,Urea Nitrogen,16.00,mg/dL
2093899,19912102,1,-58.541393,I259,0,0,0,1,0,0,Urea Nitrogen,19.00,mg/dL
2093900,19912102,1,-58.541393,I259,0,0,0,1,0,0,White Blood Cells,4.00,K/uL


In [103]:
lab_columns = ['Platelet Count', 'Red Blood Cells', 'Urea Nitrogen','Monocytes',

       'Alanine Aminotransferase (ALT)', 'Basophils', 'Eosinophils',

       'Bilirubin, Total', 'Albumin', 'Lactate Dehydrogenase (LD)',

       'Metamyelocytes', 'Atypical Lymphocytes',

        'Blasts','Promyelocytes','Cholesterol, Total', 'NTproBNP',

        '% Hemoglobin A1c', 'Glucose','C-Reactive Protein']

In [104]:
lab_columns_lower = [lab.lower() for lab in lab_columns]

diag_cml_lab = diag_cml_lab[diag_cml_lab['label'].str.lower().isin(lab_columns_lower)]


In [105]:
diag_cml_lab = diag_cml_lab.drop(columns=['icd_code', 'has_cvd'], axis=1)
diag_cml_lab = diag_cml_lab.reset_index(drop=True)

In [106]:
test_uom = diag_cml_lab[['label', 'valueuom']].drop_duplicates()
test_uom = test_uom.reset_index(drop=True)

In [107]:
test_uom

,label,valueuom
0,Metamyelocytes,%
1,Blasts,%
2,Atypical Lymphocytes,%
3,Promyelocytes,%
4,Alanine Aminotransferase (ALT),IU/L
5,Albumin,g/dL
6,Basophils,%
7,"Bilirubin, Total",mg/dL
8,Eosinophils,%
9,Glucose,mg/dL


In [108]:
diag_cml_lab = diag_cml_lab.drop('valueuom', axis=1)

In [109]:
diag_cml_lab

,subject_id,duration_cvd,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary,label,valuenum
0,10035631,0.000000,0,0,0,1,0,0,Metamyelocytes,1.00
1,10035631,0.000000,0,0,0,1,0,0,Metamyelocytes,0.00
2,10035631,0.000000,0,0,0,1,0,0,Blasts,4.00
3,10035631,0.000000,0,0,0,1,0,0,Atypical Lymphocytes,0.00
4,10035631,0.000000,0,0,0,1,0,0,Atypical Lymphocytes,2.00
...,...,...,...,...,...,...,...,...,...,...
580555,19912102,-58.541393,0,0,0,1,0,0,Platelet Count,262.00
580556,19912102,-58.541393,0,0,0,1,0,0,Red Blood Cells,3.03
580557,19912102,-58.541393,0,0,0,1,0,0,Red Blood Cells,3.14
580558,19912102,-58.541393,0,0,0,1,0,0,Urea Nitrogen,16.00


In [110]:
lab_items = diag_cml_lab['label'].unique()


lab_table_extend = diag_cml_lab.copy()


data_patient_lab = pd.DataFrame()
data_patient_lab['subject_id'] = diag_cml_lab['subject_id'].unique()


for test in lab_items:
    lab_table_extend[test] = np.where(
        lab_table_extend['label'] == test,
        lab_table_extend['valuenum'],
        np.nan
    )
    
    lab_table_extend[test] = pd.to_numeric(lab_table_extend[test])

    test_avg = lab_table_extend.groupby('subject_id')[test].mean().reset_index()
    data_patient_lab = data_patient_lab.merge(test_avg, on='subject_id', how='left')

In [111]:
data_patient_lab

,subject_id,Metamyelocytes,Blasts,Atypical Lymphocytes,Promyelocytes,Alanine Aminotransferase (ALT),Albumin,Basophils,"Bilirubin, Total",Eosinophils,Glucose,"Cholesterol, Total",C-Reactive Protein,Lactate Dehydrogenase (LD),Monocytes,Platelet Count,Red Blood Cells,Urea Nitrogen,NTproBNP,% Hemoglobin A1c
0,10035631,6.111111,34.720000,2.666667,4.857143,59.254237,3.161111,0.920000,1.216667,7.060526,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN
1,10156269,4.875000,3.750000,3.571429,4.750000,58.836066,3.666667,1.606250,0.633333,6.504000,125.883117,283.0,138.7,239.087912,10.730357,160.387097,2.997590,32.750000,35348.0,6.20
2,10207476,0.500000,NaN,2.800000,NaN,34.500000,3.178571,0.675000,0.614286,2.805128,158.612613,NaN,NaN,258.763636,6.413208,246.368421,3.374522,16.653846,615.0,6.20
3,10213803,4.666667,NaN,4.777778,1.500000,52.772727,3.460000,0.590000,0.522222,5.980769,115.970588,203.5,NaN,250.056604,8.868421,166.151899,3.345062,15.750000,NaN,NaN
4,10234917,3.166667,36.852941,3.000000,1.000000,25.068966,3.204762,0.857143,0.611111,1.844444,126.111111,133.0,NaN,342.974026,4.655556,86.444444,2.984521,30.437500,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,19788583,0.500000,64.500000,0.000000,1.000000,33.500000,4.250000,0.000000,1.100000,0.000000,149.333333,NaN,NaN,787.333333,9.000000,22.000000,2.045000,11.333333,NaN,NaN
533,19821558,2.000000,3.500000,1.500000,1.500000,28.000000,3.187500,0.690909,0.770000,3.727273,213.279412,NaN,NaN,291.311111,14.449057,84.095238,2.802523,28.307692,2100.2,9.05
534,19824742,NaN,NaN,NaN,NaN,13.000000,NaN,NaN,0.500000,NaN,102.000000,NaN,NaN,NaN,NaN,138.000000,4.350000,16.000000,NaN,NaN
535,19880967,5.777778,31.724138,2.500000,1.500000,41.235294,2.920000,1.592308,1.133333,4.523810,199.708333,107.0,NaN,492.603774,17.121569,75.675325,2.490706,19.241379,NaN,8.50


In [112]:
patient_info = diag_cml_lab.drop_duplicates(subset='subject_id')

In [113]:
full_lab = patient_info.merge(data_patient_lab, on='subject_id', how='left')

In [114]:
full_lab

,subject_id,duration_cvd,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary,label,valuenum,...,Glucose,"Cholesterol, Total",C-Reactive Protein,Lactate Dehydrogenase (LD),Monocytes,Platelet Count,Red Blood Cells,Urea Nitrogen,NTproBNP,% Hemoglobin A1c
0,10035631,0.000000,0,0,0,1,0,0,Metamyelocytes,1.0,...,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN
1,10156269,73.423127,0,0,1,0,0,0,Metamyelocytes,2.0,...,125.883117,283.0,138.7,239.087912,10.730357,160.387097,2.997590,32.750000,35348.0,6.20
2,10207476,-141.031537,0,0,0,0,0,1,Metamyelocytes,1.0,...,158.612613,NaN,NaN,258.763636,6.413208,246.368421,3.374522,16.653846,615.0,6.20
3,10213803,-55.978975,0,0,0,1,0,0,Metamyelocytes,0.0,...,115.970588,203.5,NaN,250.056604,8.868421,166.151899,3.345062,15.750000,NaN,NaN
4,10234917,0.000000,0,0,0,1,0,0,Metamyelocytes,1.0,...,126.111111,133.0,NaN,342.974026,4.655556,86.444444,2.984521,30.437500,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,19788583,0.000000,0,0,0,0,1,0,Blasts,49.0,...,149.333333,NaN,NaN,787.333333,9.000000,22.000000,2.045000,11.333333,NaN,NaN
533,19821558,-17.871222,0,0,0,1,0,0,Atypical Lymphocytes,3.0,...,213.279412,NaN,NaN,291.311111,14.449057,84.095238,2.802523,28.307692,2100.2,9.05
534,19824742,0.000000,0,0,1,0,0,0,Glucose,102.0,...,102.000000,NaN,NaN,NaN,NaN,138.000000,4.350000,16.000000,NaN,NaN
535,19880967,-25.295664,0,0,1,0,0,0,Atypical Lymphocytes,0.0,...,199.708333,107.0,NaN,492.603774,17.121569,75.675325,2.490706,19.241379,NaN,8.50


In [115]:
doses = pd.read_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\drug_doses.csv')
doses

,subject_id,drug,dose_val_rx,dose_unit_rx
0,10019003,ruxolitinib,5.0,mg
1,10019003,ruxolitinib,5.0,mg
2,10019003,ruxolitinib,5.0,mg
3,10019003,ruxolitinib,5.0,mg
4,10019003,ruxolitinib,5.0,mg
...,...,...,...,...
21153,19940147,nilotinib,150.0,mg
21154,19940147,nilotinib,300.0,mg
21155,19940147,nilotinib,400.0,mg
21156,19940147,nilotinib,150.0,mg


In [116]:
pharm = pd.read_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\pharm')
pharm = pharm.drop(columns= ['Unnamed: 0', 'frequency', 'icd_code', 'has_cvd'], axis = 1)
pharm

,subject_id,duration_cvd,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary,doses_per_24_hrs,duration_med,duration_interval,prescribed_dose,Cabozantinib,Crizotinib,Dasatinib,Imatinib,Nilotinib,Ruxolitinib
0,10035631,0.000000,0,0,0,1,0,0,3,NaN,Ongoing,NaN,0,0,0,0,0,0
1,10035631,0.000000,0,0,0,1,0,0,2,NaN,Ongoing,NaN,0,0,0,0,0,0
2,10035631,0.000000,0,0,0,1,0,0,1,NaN,Ongoing,NaN,0,0,0,0,0,0
3,10035631,0.000000,0,0,0,1,0,0,1,5.0,Days,NaN,0,0,0,0,0,0
4,10035631,0.000000,0,0,0,1,0,0,1,3.0,Days,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49512,19912102,-58.541393,0,0,0,1,0,0,1,1.0,Doses,NaN,0,0,0,0,0,0
49513,19912102,-58.541393,0,0,0,1,0,0,1,NaN,Ongoing,NaN,0,0,0,0,0,0
49514,19912102,-58.541393,0,0,0,1,0,0,2,NaN,Ongoing,NaN,0,0,0,0,0,0
49515,19912102,-58.541393,0,0,0,1,0,0,3,2.0,Days,NaN,0,0,0,0,0,0


In [117]:
diag_total = pharm.merge(full_lab, on='subject_id')

In [120]:
diag_total = diag_total.drop(columns=[
    'duration_cvd_y', 'arterioles_y', 'cerebrovascular_y',
    'hypertensive_y', 'ischemic_y', 'other_y', 'pulmonary_y'
])


diag_total.rename(columns={
    'duration_cvd_x': 'duration_cvd',
    'arterioles_x': 'arterioles',
    'cerebrovascular_x': 'cerebrovascular',
    'hypertensive_x': 'hypertensive',
    'ischemic_x': 'ischemic',
    'other_x': 'other',
    'pulmonary_x': 'pulmonary'
}, inplace=True)

In [121]:
diag_total

,subject_id,duration_cvd,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary,doses_per_24_hrs,duration_med,...,Glucose,"Cholesterol, Total",C-Reactive Protein,Lactate Dehydrogenase (LD),Monocytes,Platelet Count,Red Blood Cells,Urea Nitrogen,NTproBNP,% Hemoglobin A1c
0,10035631,0.000000,0,0,0,1,0,0,3,NaN,...,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN
1,10035631,0.000000,0,0,0,1,0,0,2,NaN,...,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN
2,10035631,0.000000,0,0,0,1,0,0,1,NaN,...,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN
3,10035631,0.000000,0,0,0,1,0,0,1,5.0,...,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN
4,10035631,0.000000,0,0,0,1,0,0,1,3.0,...,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49506,19912102,-58.541393,0,0,0,1,0,0,1,1.0,...,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN
49507,19912102,-58.541393,0,0,0,1,0,0,1,NaN,...,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN
49508,19912102,-58.541393,0,0,0,1,0,0,2,NaN,...,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN
49509,19912102,-58.541393,0,0,0,1,0,0,3,2.0,...,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN


In [ ]:
diag_total.to_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\diag_total.csv')

In [124]:
omr_data = pd.read_csv(r'C:\Users\Yahya\Documents\GitHub\Internship\CSV_files\omr_table.csv')

c:\Users\Yahya\Documents\GitHub\Internship\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
omr_data['result_value'] = omr_data['result_value'].astype(str)

bmi_data = omr_data[omr_data['result_name'].str.lower().isin(['bmi', 'bmi (kg/m2)'])].copy()

bmi_data['result_value'] = pd.to_numeric(bmi_data['result_value'], errors='coerce')

mean_bmi = bmi_data.groupby('subject_id')['result_value'].mean().reset_index(name='mean_bmi')

bp_data = omr_data[omr_data['result_name'].str.lower() == 'blood pressure'].copy()

bp_split = bp_data['result_value'].str.extract(r'(?P<systolic>\d+)[^\d]+(?P<diastolic>\d+)')
bp_split = bp_split.astype(float)
bp_data = bp_data.assign(systolic=bp_split['systolic'], diastolic=bp_split['diastolic'])

mean_bp = bp_data.groupby('subject_id')[['systolic', 'diastolic']].mean().reset_index()
mean_bp.rename(columns={'systolic': 'mean_systolic', 'diastolic': 'mean_diastolic'}, inplace=True)

final_omr = pd.merge(mean_bmi, mean_bp, on='subject_id', how='outer')

In [138]:
diag_final = diag_total.merge(final_omr, on='subject_id')

In [140]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [141]:
diag_final

,subject_id,duration_cvd,arterioles,cerebrovascular,hypertensive,ischemic,other,pulmonary,doses_per_24_hrs,duration_med,duration_interval,prescribed_dose,Cabozantinib,Crizotinib,Dasatinib,Imatinib,Nilotinib,Ruxolitinib,label,valuenum,Metamyelocytes,Blasts,Atypical Lymphocytes,Promyelocytes,Alanine Aminotransferase (ALT),Albumin,Basophils,"Bilirubin, Total",Eosinophils,Glucose,"Cholesterol, Total",C-Reactive Protein,Lactate Dehydrogenase (LD),Monocytes,Platelet Count,Red Blood Cells,Urea Nitrogen,NTproBNP,% Hemoglobin A1c,mean_bmi,mean_systolic,mean_diastolic
0,10035631,0.000000,0,0,0,1,0,0,3,NaN,Ongoing,NaN,0,0,0,0,0,0,Metamyelocytes,1.0,6.111111,34.720000,2.666667,4.857143,59.254237,3.161111,0.920000,1.216667,7.060526,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN,24.939130,112.145833,64.989583
1,10035631,0.000000,0,0,0,1,0,0,2,NaN,Ongoing,NaN,0,0,0,0,0,0,Metamyelocytes,1.0,6.111111,34.720000,2.666667,4.857143,59.254237,3.161111,0.920000,1.216667,7.060526,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN,24.939130,112.145833,64.989583
2,10035631,0.000000,0,0,0,1,0,0,1,NaN,Ongoing,NaN,0,0,0,0,0,0,Metamyelocytes,1.0,6.111111,34.720000,2.666667,4.857143,59.254237,3.161111,0.920000,1.216667,7.060526,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN,24.939130,112.145833,64.989583
3,10035631,0.000000,0,0,0,1,0,0,1,5.0,Days,NaN,0,0,0,0,0,0,Metamyelocytes,1.0,6.111111,34.720000,2.666667,4.857143,59.254237,3.161111,0.920000,1.216667,7.060526,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN,24.939130,112.145833,64.989583
4,10035631,0.000000,0,0,0,1,0,0,1,3.0,Days,NaN,0,0,0,0,0,0,Metamyelocytes,1.0,6.111111,34.720000,2.666667,4.857143,59.254237,3.161111,0.920000,1.216667,7.060526,149.176471,99.0,139.3,1666.404959,14.868085,172.494845,2.924455,50.673469,NaN,NaN,24.939130,112.145833,64.989583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48466,19912102,-58.541393,0,0,0,1,0,0,1,1.0,Doses,NaN,0,0,0,0,0,0,Blasts,5.0,1.500000,4.285714,1.000000,NaN,45.928571,4.033333,6.481818,0.300000,3.028571,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN,24.836585,124.278481,72.544304
48467,19912102,-58.541393,0,0,0,1,0,0,1,NaN,Ongoing,NaN,0,0,0,0,0,0,Blasts,5.0,1.500000,4.285714,1.000000,NaN,45.928571,4.033333,6.481818,0.300000,3.028571,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN,24.836585,124.278481,72.544304
48468,19912102,-58.541393,0,0,0,1,0,0,2,NaN,Ongoing,NaN,0,0,0,0,0,0,Blasts,5.0,1.500000,4.285714,1.000000,NaN,45.928571,4.033333,6.481818,0.300000,3.028571,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN,24.836585,124.278481,72.544304
48469,19912102,-58.541393,0,0,0,1,0,0,3,2.0,Days,NaN,0,0,0,0,0,0,Blasts,5.0,1.500000,4.285714,1.000000,NaN,45.928571,4.033333,6.481818,0.300000,3.028571,99.705882,NaN,5.9,454.947368,4.733333,909.413793,3.157600,18.642857,NaN,NaN,24.836585,124.278481,72.544304


In [142]:
cvd_columns = ['arterioles', 'cerebrovascular', 'hypertensive', 'ischemic', 'other', 'pulmonary']

no_cvd_count = (diag_final[cvd_columns].sum(axis=1) == 0).sum()

print(f"Number of patients with no CVD: {no_cvd_count}")

Number of patients with no CVD: 0
